In [1]:
dbutils.library.installPyPI("mlflow", "1.0.0")

Out[1]: True

In [2]:
import mlflow.sklearn
import tempfile

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score

Tue May 12 03:54:30 2020 py4j imported

## Load Data

In [4]:
#w q1 train sets
pd_train_w_q1_wo_resamp = spark.read.csv('/mnt/xql2001-gr5069/processed/final_project/modelsets/spark_pred_train_w_q1_wo_resamp.csv',header=True).toPandas()
pd_train_w_q1_undersamp = spark.read.csv('/mnt/xql2001-gr5069/processed/final_project/modelsets/spark_pred_train_w_q1_undersamp.csv',header=True).toPandas()
pd_train_w_q1_oversamp = spark.read.csv('/mnt/xql2001-gr5069/processed/final_project/modelsets/spark_pred_train_w_q1_oversamp.csv',header=True).toPandas()
pd_train_w_q1_smote = spark.read.csv('/mnt/xql2001-gr5069/processed/final_project/modelsets/spark_pred_train_w_q1_smote.csv',header=True).toPandas()

#w/o q1 train sets
pd_train_wo_q1_wo_resamp = spark.read.csv('/mnt/xql2001-gr5069/processed/final_project/modelsets/spark_pred_train_wo_q1_wo_resamp.csv',header=True).toPandas()
pd_train_wo_q1_undersamp = spark.read.csv('/mnt/xql2001-gr5069/processed/final_project/modelsets/spark_pred_train_wo_q1_undersamp.csv',header=True).toPandas()
pd_train_wo_q1_oversamp = spark.read.csv('/mnt/xql2001-gr5069/processed/final_project/modelsets/spark_pred_train_wo_q1_oversamp.csv',header=True).toPandas()
pd_train_wo_q1_smote = spark.read.csv('/mnt/xql2001-gr5069/processed/final_project/modelsets/spark_pred_train_wo_q1_smote.csv',header=True).toPandas()

#test sets
pd_test_w_q1 = spark.read.csv('/mnt/xql2001-gr5069/processed/final_project/modelsets/spark_pred_test_w_q1.csv',header=True).toPandas()
pd_test_wo_q1 = spark.read.csv('/mnt/xql2001-gr5069/processed/final_project/modelsets/spark_pred_test_wo_q1.csv',header=True).toPandas()

In [5]:
# test set dfs to work with
pd_test_w_q1_working = pd_test_w_q1.copy(deep = True)
pd_test_w_q1_working['Second'] = pd_test_w_q1_working['Second'].astype(int)
pd_test_wo_q1_working = pd_test_wo_q1.copy(deep = True)
pd_test_wo_q1_working['Second'] = pd_test_wo_q1_working['Second'].astype(int)

# Test sets to give sklearn
pd_test_w_q1_4_predict = pd_test_w_q1.drop(['raceId', 'driverId', 'constructorId', 'Second'], axis = 1)
pd_test_wo_q1_4_predict = pd_test_wo_q1.drop(['raceId', 'driverId', 'constructorId', 'Second'], axis = 1)

## ML Flow Setup

In [7]:
def log_rf(run_name, params, train, test_4_model, test_w_Ids):
  # :::::::::::: DESCRIPTION ::::::::::::
  # This function is used to fit a Random Forest model to the F1 data
  # and generate  predicted second place positions based on the most 
  # likely second place finish for each race.
  #
  # Model parameters and feature importances, are also logged in MLFlow
  #
  # ::::::::: INPUTS ::::::::::::
  # 1. run_name - A string, the name of the run registered in MLFlow
  # 2. Params - A dictionary of the parameters to pass to the model in 
  # the form {'parameter': value}
  # 3. train - The training data, to pass to model.fit() for an Sklearn 
  # model object
  # 4. test_4_model - The test data, to pass to model.predict_proba()
  # for an Sklearn model object. Should only contain features, without 
  # reference Ids.
  # 5. test_w_Ids - The test data with Ids, to group predicted probabilities
  # by raceId and compute the most likely second place finish in each race.
  #
  # ::::::::: OUTPUT ::::::::::::
  # Prints Precision Score
  # Model, parameters, and feature importance logged in MLFlow
  
  with mlflow.start_run(run_name = run_name) as run:
    runID = run.info.run_uuid
    experimentID = run.info.experiment_id
    
    # Create model, train it
    model = RandomForestClassifier(**params)
    model.fit(train.drop(['Second'], axis = 1),
              train[['Second']].values.ravel())
    
    # Log model
    # mlflow.sklearn.log_model(model, "random-forest-model")
    
    # Log params
    [mlflow.log_param(param, value) for param, value in params.items()]
    
    # Prediction
    test_w_Ids['predict_proba'] = model.predict_proba(test_4_model)[:, 1]
    max_prob_per_race = pd.DataFrame(test_w_Ids.groupby(['raceId']).predict_proba.max())
    test_w_Ids = test_w_Ids\
      .merge(max_prob_per_race, on = 'raceId', how = 'inner', suffixes = ('_driver', '_max'))
    
    # Classify driver with max probability for that race as 1
    test_w_Ids['pred'] = test_w_Ids['predict_proba_driver']\
      .eq(test_w_Ids['predict_proba_max']).astype(int)

    # Create metrics
    precision = precision_score(test_w_Ids['Second'], test_w_Ids['pred'])
    
    # Print metrics
    print("precision: {}".format(precision))
    
    # Log metrics
    mlflow.log_metric("precision", precision)
    
    # Create feature importance
    importance = pd.DataFrame(list(zip(train.drop(['Second'], axis = 1).columns,
                                       model.feature_importances_)), 
                                columns=["Feature", "Importance"]
                              ).sort_values("Importance", ascending=False)
    
    # Log importances using a temporary file
    temp = tempfile.NamedTemporaryFile(prefix="feature-importance-", suffix=".csv")
    temp_name = temp.name
    try:
      importance.to_csv(temp_name, index=False)
      mlflow.log_artifact(temp_name, "feature-importance.csv")
    finally:
      temp.close() # Delete the temp file

    return run.info.run_uuid

In [8]:
def log_gb(run_name, params, train, test_4_model, test_w_Ids):
  # :::::::::::: DESCRIPTION ::::::::::::
  # This function is used to fit a Random Forest model to the F1 data
  # and generate  predicted second place positions based on the most 
  # likely second place finish for each race.
  #
  # Model parameters and feature importances, are also logged in MLFlow
  #
  # ::::::::: INPUTS ::::::::::::
  # 1. run_name - A string, the name of the run registered in MLFlow
  # 2. Params - A dictionary of the parameters to pass to the model in 
  # the form {'parameter': value}
  # 3. train - The training data, to pass to model.fit() for an Sklearn 
  # model object
  # 4. test_4_model - The test data, to pass to model.predict_proba()
  # for an Sklearn model object. Should only contain features, without 
  # reference Ids.
  # 5. test_w_Ids - The test data with Ids, to group predicted probabilities
  # by raceId and compute the most likely second place finish in each race.
  #
  # ::::::::: OUTPUT ::::::::::::
  # Prints Precision Score
  # Model, parameters, and feature importance logged in MLFlow
  
  with mlflow.start_run(run_name = run_name) as run:
    runID = run.info.run_uuid
    experimentID = run.info.experiment_id
    
    # Create model, train it
    model = GradientBoostingClassifier(**params)
    model.fit(train.drop(['Second'], axis = 1),
              train[['Second']].values.ravel())
    
    # Log model
    # mlflow.sklearn.log_model(model, "random-forest-model")
    
    # Log params
    [mlflow.log_param(param, value) for param, value in params.items()]
    
    # Prediction
    test_w_Ids['predict_proba'] = model.predict_proba(test_4_model)[:, 1]
    max_prob_per_race = pd.DataFrame(test_w_Ids.groupby(['raceId']).predict_proba.max())
    test_w_Ids = test_w_Ids\
      .merge(max_prob_per_race, on = 'raceId', how = 'inner', suffixes = ('_driver', '_max'))
    
    # Classify driver with max probability for that race as 1
    test_w_Ids['pred'] = test_w_Ids['predict_proba_driver']\
      .eq(test_w_Ids['predict_proba_max']).astype(int)

    # Create metrics
    precision = precision_score(test_w_Ids['Second'], test_w_Ids['pred'])
    
    # Print metrics
    print("precision: {}".format(precision))
    
    # Log metrics
    mlflow.log_metric("precision", precision)
    
    # Create feature importance
    importance = pd.DataFrame(list(zip(train.drop(['Second'], axis = 1).columns,
                                       model.feature_importances_)), 
                                columns=["Feature", "Importance"]
                              ).sort_values("Importance", ascending=False)
    
    # Log importances using a temporary file
    temp = tempfile.NamedTemporaryFile(prefix="feature-importance-", suffix=".csv")
    temp_name = temp.name
    try:
      importance.to_csv(temp_name, index=False)
      mlflow.log_artifact(temp_name, "feature-importance.csv")
    finally:
      temp.close() # Delete the temp file

    return run.info.run_uuid

### Testing MLFlow implementation

In [10]:
params_test_rf = {'n_estimators': 1000,
                  'max_depth': 5,
                  'max_features': 'sqrt',
                  'random_state': 123
                 }

log_rf('test run rf', params_test_rf, pd_train_w_q1_wo_resamp, pd_test_w_q1_4_predict, pd_test_w_q1_working)

precision: 0.15328467153284672
Out[22]: '9052ec3ec7e640d0a6803ab2ccf488fc'

In [11]:
params_test_gb = {'loss': 'deviance',
                  'learning_rate': 0.1,
                  'n_estimators': 1000,
                  'max_depth': 5,
                  'max_features': 'sqrt',
                  'random_state': 123
                 }

log_gb('test run gb', params_test_gb, pd_train_w_q1_wo_resamp, pd_test_w_q1_4_predict, pd_test_w_q1_working)

precision: 0.17518248175182483
Out[25]: 'ae18f9e405a241338e2673ff1f05d8de'

#### It works!

## Model Runs

### No Resampling, w/ q1 data 

**RF**

* n_estimators - 1000
* max_depth - 1, 2, 3, 4, 5, 6, 7, 8, 9, 10
* max_features - 'sqrt', None
* random_state - 123

**GB**
* loss - 'deviance'
* learning_rate - 0.3, 0.5, 0.7, 0.9
* n_estimators - 1000
* max_depth - 1, 2, 3, 4, 5, 6, 7, 8, 9, 10
* max_features - 'sqrt', None
* random_state - 123

In [15]:
for depth in np.arange(1, 11):
  for feature_no in ['sqrt', None]:
    params_rf_w_q1_wo_resamp = {'n_estimators': 1000,
                                'max_depth': depth,
                                'max_features': feature_no,
                                'random_state': 123
                               }
    log_rf('rf_w_q1_wo_resamp',
           params_rf_w_q1_wo_resamp,
           pd_train_w_q1_wo_resamp,
           pd_test_w_q1_4_predict,
           pd_test_w_q1_working)

precision: 0.20320855614973263
precision: 0.1882998171846435
precision: 0.145985401459854
precision: 0.17518248175182483
precision: 0.11678832116788321
precision: 0.17518248175182483
precision: 0.15328467153284672
precision: 0.2116788321167883
precision: 0.15328467153284672
precision: 0.19708029197080293
precision: 0.15328467153284672
precision: 0.1897810218978102
precision: 0.145985401459854
precision: 0.18248175182481752
precision: 0.145985401459854
precision: 0.19708029197080293
precision: 0.16058394160583941
precision: 0.2116788321167883
precision: 0.1897810218978102
precision: 0.21897810218978103

In [16]:
for depth in np.arange(1, 11):
  for learning_rate in [0.3, 0.5, 0.7, 0.9]:
    for feature_no in ['sqrt', None]:
      params_gb_w_q1_wo_resamp = {'loss': 'deviance',
                                  'learning_rate': learning_rate,
                                  'n_estimators': 1000,
                                  'max_depth': depth,
                                  'max_features': feature_no,
                                  'random_state': 123
                                 }
      log_gb('gb_w_q1_wo_resamp',
             params_gb_w_q1_wo_resamp,
             pd_train_w_q1_wo_resamp,
             pd_test_w_q1_4_predict,
             pd_test_w_q1_working)

precision: 0.13380281690140844
precision: 0.20982142857142858
precision: 0.13138686131386862
precision: 0.12162162162162163
precision: 0.1678832116788321
precision: 0.12162162162162163
precision: 0.1619718309859155
precision: 0.176759410801964
precision: 0.17518248175182483
precision: 0.16058394160583941
precision: 0.145985401459854
precision: 0.15328467153284672
precision: 0.15328467153284672
precision: 0.18248175182481752
precision: 0.15714285714285714
precision: 0.147239263803681
precision: 0.18248175182481752
precision: 0.1897810218978102
precision: 0.16633266533066132
precision: 0.13138686131386862
precision: 0.07692307692307693
precision: 0.16058394160583941
precision: 0.038869257950530034
precision: 0.15172413793103448
precision: 0.09137055837563451
precision: 0.12408759124087591
precision: 0.08596837944664032
precision: 0.2014388489208633
precision: 0.08379888268156424
precision: 0.18248175182481752
precision: 0.08216783216783216
precision: 0.19285714285714287
precision: 0.0948905109489051
precision: 0.13138686131386862
precision: 0.03115264797507788
precision: 0.11594202898550725
precision: 0.0620782726045884
precision: 0.07194244604316546
precision: 0.01788617886178862
precision: 0.03783783783783784
precision: 0.053830227743271224
precision: 0.13138686131386862
precision: 0.16058394160583941
precision: 0.13043478260869565
precision: 0.08023483365949119
precision: 0.07602339181286549
precision: 0.06502890173410404
precision: 0.10071942446043165
precision: 0.1386861313868613
precision: 0.12408759124087591
precision: 0.11678832116788321
precision: 0.06293706293706294
precision: 0.09726027397260274
precision: 0.09271523178807947
precision: 0.10181311018131102
precision: 0.02050113895216401
precision: 0.1678832116788321
precision: 0.1386861313868613
precision: 0.07005253940455342
precision: 0.05421686746987952
precision: 0.054945054945054944
precision: 0.0896551724137931
precision: 0.09170731707317073
precision: 0.1111111111111111
precision: 0.16058394160583941
precision: 0.13138686131386862
precision: 0.050352467270896276
precision: 0.12408759124087591
precision: 0.04417670682730924
precision: 0.06206896551724138
precision: 0.018691588785046728
precision: 0.06349206349206349
precision: 0.145985401459854
precision: 0.12408759124087591
precision: 0.07321131447587355
precision: 0.12318840579710146
precision: 0.061971830985915494
precision: 0.06557377049180328
precision: 0.08866995073891626
precision: 0.08300395256916997

### Undersampling, w/ q1 data 

**RF**

* n_estimators - 1000
* max_depth - 1, 2, 3, 4, 5, 6, 7, 8, 9, 10
* max_features - 'sqrt', None
* random_state - 123

**GB**
* loss - 'deviance'
* learning_rate - 0.3, 0.5, 0.7, 0.9
* n_estimators - 1000
* max_depth - 1, 2, 3, 4, 5, 6, 7, 8, 9, 10
* max_features - 'sqrt', None
* random_state - 123

In [18]:
for depth in np.arange(1, 11):
  for feature_no in ['sqrt', None]:
    params_rf_w_q1_undersamp = {'n_estimators': 1000,
                                'max_depth': depth,
                                'max_features': feature_no,
                                'random_state': 123
                               }
    log_rf('rf_w_q1_undersamp',
           params_rf_w_q1_undersamp,
           pd_train_w_q1_undersamp,
           pd_test_w_q1_4_predict,
           pd_test_w_q1_working)

precision: 0.15952732644017725
precision: 0.146163215590743
precision: 0.23225806451612904
precision: 0.203125
precision: 0.145985401459854
precision: 0.18248175182481752
precision: 0.17518248175182483
precision: 0.2116788321167883
precision: 0.19708029197080293
precision: 0.1678832116788321
precision: 0.1897810218978102
precision: 0.15328467153284672
precision: 0.1897810218978102
precision: 0.15328467153284672
precision: 0.15328467153284672
precision: 0.16058394160583941
precision: 0.15328467153284672
precision: 0.16058394160583941
precision: 0.13138686131386862
precision: 0.145985401459854

In [19]:
for depth in np.arange(1, 11):
  for learning_rate in [0.3, 0.5, 0.7, 0.9]:
    for feature_no in ['sqrt', None]:
      params_gb_w_q1_undersamp = {'loss': 'deviance',
                                  'learning_rate': learning_rate,
                                  'n_estimators': 1000,
                                  'max_depth': depth,
                                  'max_features': feature_no,
                                  'random_state': 123
                                 }
      log_gb('gb_w_q1_undersamp',
             params_gb_w_q1_undersamp,
             pd_train_w_q1_undersamp,
             pd_test_w_q1_4_predict,
             pd_test_w_q1_working)

precision: 0.13768115942028986
precision: 0.13043478260869565
precision: 0.13768115942028986
precision: 0.12318840579710146
precision: 0.15942028985507245
precision: 0.13768115942028986
precision: 0.1678832116788321
precision: 0.16666666666666666
precision: 0.10218978102189781
precision: 0.1678832116788321
precision: 0.1386861313868613
precision: 0.15328467153284672
precision: 0.145985401459854
precision: 0.16058394160583941
precision: 0.13138686131386862
precision: 0.15217391304347827
precision: 0.17518248175182483
precision: 0.13138686131386862
precision: 0.16058394160583941
precision: 0.1386861313868613
precision: 0.11678832116788321
precision: 0.11678832116788321
precision: 0.1386861313868613
precision: 0.12408759124087591
precision: 0.1386861313868613
precision: 0.12408759124087591
precision: 0.13138686131386862
precision: 0.145985401459854
precision: 0.145985401459854
precision: 0.1386861313868613
precision: 0.20437956204379562
precision: 0.18248175182481752
precision: 0.1678832116788321
precision: 0.1386861313868613
precision: 0.11678832116788321
precision: 0.1678832116788321
precision: 0.13138686131386862
precision: 0.15217391304347827
precision: 0.09859154929577464
precision: 0.11888111888111888
precision: 0.13138686131386862
precision: 0.17518248175182483
precision: 0.15328467153284672
precision: 0.17142857142857143
precision: 0.11387559808612441
precision: 0.13245033112582782
precision: 0.14583333333333334
precision: 0.13523131672597866
precision: 0.10948905109489052
precision: 0.1386861313868613
precision: 0.11594202898550725
precision: 0.15853658536585366
precision: 0.13756613756613756
precision: 0.16666666666666666
precision: 0.15057915057915058
precision: 0.14479638009049775
precision: 0.16666666666666666
precision: 0.18055555555555555
precision: 0.1306532663316583
precision: 0.15763546798029557
precision: 0.16470588235294117
precision: 0.1652542372881356
precision: 0.13733905579399142
precision: 0.14324324324324325
precision: 0.1503267973856209
precision: 0.1927710843373494
precision: 0.13705583756345177
precision: 0.15979381443298968
precision: 0.12727272727272726
precision: 0.13541666666666666
precision: 0.1375
precision: 0.16964285714285715
precision: 0.13636363636363635
precision: 0.18229166666666666
precision: 0.11734693877551021
precision: 0.16216216216216217
precision: 0.13270142180094788
precision: 0.1377551020408163
precision: 0.10606060606060606
precision: 0.11702127659574468

### Oversampling, w/ q1 data 

**RF**

* n_estimators - 1000
* max_depth - 1, 2, 3, 4, 5, 6, 7, 8, 9, 10
* max_features - 'sqrt', None
* random_state - 123

**GB**
* loss - 'deviance'
* learning_rate - 0.3, 0.5, 0.7, 0.9
* n_estimators - 1000
* max_depth - 1, 2, 3, 4, 5, 6, 7, 8, 9, 10
* max_features - 'sqrt', None
* random_state - 123

In [21]:
for depth in np.arange(1, 11):
  for feature_no in ['sqrt', None]:
    params_rf_w_q1_oversamp = {'n_estimators': 1000,
                               'max_depth': depth,
                               'max_features': feature_no,
                               'random_state': 123
                              }
    log_rf('rf_w_q1_oversamp',
           params_rf_w_q1_oversamp,
           pd_train_w_q1_oversamp,
           pd_test_w_q1_4_predict,
           pd_test_w_q1_working)

precision: 0.13078703703703703
precision: 0.10633116883116883
precision: 0.21019108280254778
precision: 0.1882998171846435
precision: 0.1337579617834395
precision: 0.19405940594059407
precision: 0.1386861313868613
precision: 0.22289156626506024
precision: 0.1897810218978102
precision: 0.21637426900584794
precision: 0.20437956204379562
precision: 0.2246376811594203
precision: 0.19708029197080293
precision: 0.17518248175182483
precision: 0.20437956204379562
precision: 0.1897810218978102
precision: 0.20437956204379562
precision: 0.18248175182481752
precision: 0.20437956204379562
precision: 0.1678832116788321

In [22]:
for depth in np.arange(1, 11):
  for learning_rate in [0.3, 0.5, 0.7, 0.9]:
    for feature_no in ['sqrt', None]:
      params_gb_w_q1_oversamp = {'loss': 'deviance',
                                 'learning_rate': learning_rate,
                                 'n_estimators': 1000,
                                 'max_depth': depth,
                                 'max_features': feature_no,
                                 'random_state': 123
                                }
      log_gb('gb_w_q1_oversamp',
             params_gb_w_q1_oversamp,
             pd_train_w_q1_oversamp,
             pd_test_w_q1_4_predict,
             pd_test_w_q1_working)

precision: 0.16058394160583941
precision: 0.1678832116788321
precision: 0.18248175182481752
precision: 0.16058394160583941
precision: 0.176056338028169
precision: 0.18248175182481752
precision: 0.1897810218978102
precision: 0.1678832116788321
precision: 0.16058394160583941
precision: 0.145985401459854
precision: 0.1678832116788321
precision: 0.13138686131386862
precision: 0.1897810218978102
precision: 0.16058394160583941
precision: 0.18248175182481752
precision: 0.1386861313868613
precision: 0.1678832116788321
precision: 0.13138686131386862
precision: 0.16058394160583941
precision: 0.145985401459854
precision: 0.1678832116788321
precision: 0.11678832116788321
precision: 0.10948905109489052
precision: 0.11678832116788321
precision: 0.145985401459854
precision: 0.145985401459854
precision: 0.0948905109489051
precision: 0.12408759124087591
precision: 0.10948905109489052
precision: 0.1386861313868613
precision: 0.19708029197080293
precision: 0.12408759124087591
precision: 0.1678832116788321
precision: 0.13138686131386862
precision: 0.11678832116788321
precision: 0.12408759124087591
precision: 0.13138686131386862
precision: 0.12408759124087591
precision: 0.13138686131386862
precision: 0.10218978102189781
precision: 0.11678832116788321
precision: 0.10218978102189781
precision: 0.15328467153284672
precision: 0.1386861313868613
precision: 0.1678832116788321
precision: 0.0948905109489051
precision: 0.12408759124087591
precision: 0.13138686131386862
precision: 0.13138686131386862
precision: 0.12408759124087591
precision: 0.1678832116788321
precision: 0.10948905109489052
precision: 0.15328467153284672
precision: 0.1386861313868613
precision: 0.13138686131386862
precision: 0.10948905109489052
precision: 0.15328467153284672
precision: 0.12408759124087591
precision: 0.12408759124087591
precision: 0.12408759124087591
precision: 0.1386861313868613
precision: 0.06569343065693431
precision: 0.11678832116788321
precision: 0.13138686131386862
precision: 0.145985401459854
precision: 0.11678832116788321
precision: 0.1386861313868613
precision: 0.11678832116788321
precision: 0.12408759124087591
precision: 0.12408759124087591
precision: 0.10948905109489052
precision: 0.13138686131386862
precision: 0.145985401459854
precision: 0.13138686131386862
precision: 0.13138686131386862
precision: 0.12408759124087591
precision: 0.13138686131386862
precision: 0.13138686131386862
precision: 0.15328467153284672
precision: 0.10218978102189781

### SMOTE, w/ q1 data 

**RF**

* n_estimators - 1000
* max_depth - 1, 2, 3, 4, 5, 6, 7, 8, 9, 10
* max_features - 'sqrt', None
* random_state - 123

**GB**
* loss - 'deviance'
* learning_rate - 0.3, 0.5, 0.7, 0.9
* n_estimators - 1000
* max_depth - 1, 2, 3, 4, 5, 6, 7, 8, 9, 10
* max_features - 'sqrt', None
* random_state - 123

In [24]:
for depth in np.arange(1, 11):
  for feature_no in ['sqrt', None]:
    params_rf_w_q1_smote = {'n_estimators': 1000,
                            'max_depth': depth,
                            'max_features': feature_no,
                            'random_state': 123
                           }
    log_rf('rf_w_q1_smote',
           params_rf_w_q1_smote,
           pd_train_w_q1_smote,
           pd_test_w_q1_4_predict,
           pd_test_w_q1_working)

precision: 0.13836477987421383
precision: 0.11506849315068493
precision: 0.18064516129032257
precision: 0.1319052987598647
precision: 0.1791044776119403
precision: 0.1319052987598647
precision: 0.145985401459854
precision: 0.1891891891891892
precision: 0.145985401459854
precision: 0.1891891891891892
precision: 0.145985401459854
precision: 0.2033195020746888
precision: 0.1386861313868613
precision: 0.11678832116788321
precision: 0.1386861313868613
precision: 0.12408759124087591
precision: 0.12408759124087591
precision: 0.12408759124087591
precision: 0.13138686131386862
precision: 0.13138686131386862

In [25]:
for depth in np.arange(1, 11):
  for learning_rate in [0.3, 0.5, 0.7, 0.9]:
    for feature_no in ['sqrt', None]:
      params_gb_w_q1_smote = {'loss': 'deviance',
                              'learning_rate': learning_rate,
                              'n_estimators': 1000,
                              'max_depth': depth,
                              'max_features': feature_no,
                              'random_state': 123
                             }
      log_gb('gb_w_q1_smote',
             params_gb_w_q1_smote,
             pd_train_w_q1_smote,
             pd_test_w_q1_4_predict,
             pd_test_w_q1_working)

java.io.IOException: Connection failed
	at com.databricks.rpc.Jetty9Client$$anon$1.handleError(Jetty9Client.scala:468)
	at com.databricks.rpc.Jetty9Client$$anon$1.onFailure(Jetty9Client.scala:395)
	at shaded.v9_4.org.eclipse.jetty.client.ResponseNotifier.notifyFailure(ResponseNotifier.java:177)
	at shaded.v9_4.org.eclipse.jetty.client.ResponseNotifier.notifyFailure(ResponseNotifier.java:169)
	at shaded.v9_4.org.eclipse.jetty.client.HttpExchange.notifyFailureComplete(HttpExchange.java:268)
	at shaded.v9_4.org.eclipse.jetty.client.HttpExchange.abort(HttpExchange.java:240)
	at shaded.v9_4.org.eclipse.jetty.client.HttpConversation.abort(HttpConversation.java:149)
	at shaded.v9_4.org.eclipse.jetty.client.HttpRequest.abort(HttpRequest.java:768)
	at shaded.v9_4.org.eclipse.jetty.client.HttpDestination.abort(HttpDestination.java:453)
	at shaded.v9_4.org.eclipse.jetty.client.HttpDestination.failed(HttpDestination.java:234)
	at shaded.v9_4.org.eclipse.jetty.client.AbstractConnectionPool$1.failed(AbstractConnectionPool.java:140)
	at shaded.v9_4.org.eclipse.jetty.util.Promise$Wrapper.failed(Promise.java:136)
	at shaded.v9_4.org.eclipse.jetty.client.HttpClient$1$1.failed(HttpClient.java:612)
	at shaded.v9_4.org.eclipse.jetty.client.AbstractConnectorHttpClientTransport.connectFailed(AbstractConnectorHttpClientTransport.java:138)
	at shaded.v9_4.org.eclipse.jetty.client.AbstractConnectorHttpClientTransport$ClientSelectorManager.connectionFailed(AbstractConnectorHttpClientTransport.java:188)
	at shaded.v9_4.org.eclipse.jetty.io.ManagedSelector$Connect.failed(ManagedSelector.java:822)
	at shaded.v9_4.org.eclipse.jetty.io.ManagedSelector.processConnect(ManagedSelector.java:254)
	at shaded.v9_4.org.eclipse.jetty.io.ManagedSelector.access$1400(ManagedSelector.java:62)
	at shaded.v9_4.org.eclipse.jetty.io.ManagedSelector$SelectorProducer.processSelected(ManagedSelector.java:543)
	at shaded.v9_4.org.eclipse.jetty.io.ManagedSelector$SelectorProducer.produce(ManagedSelector.java:401)
	at shaded.v9_4.org.eclipse.jetty.util.thread.strategy.EatWhatYouKill.produceTask(EatWhatYouKill.java:360)
	at shaded.v9_4.org.eclipse.jetty.util.thread.strategy.EatWhatYouKill.doProduce(EatWhatYouKill.java:184)
	at shaded.v9_4.org.eclipse.jetty.util.thread.strategy.EatWhatYouKill.tryProduce(EatWhatYouKill.java:171)
	at shaded.v9_4.org.eclipse.jetty.util.thread.strategy.EatWhatYouKill.run(EatWhatYouKill.java:129)
	at shaded.v9_4.org.eclipse.jetty.util.thread.ReservedThreadExecutor$ReservedThread.run(ReservedThreadExecutor.java:367)
	at shaded.v9_4.org.eclipse.jetty.util.thread.QueuedThreadPool.runJob(QueuedThreadPool.java:782)
	at shaded.v9_4.org.eclipse.jetty.util.thread.QueuedThreadPool$Runner.run(QueuedThreadPool.java:914)
	at java.lang.Thread.run(Thread.java:748)
Caused by: java.net.ConnectException: Connection refused
	at sun.nio.ch.SocketChannelImpl.checkConnect(Native Method)
	at sun.nio.ch.SocketChannelImpl.finishConnect(SocketChannelImpl.java:714)
	at shaded.v9_4.org.eclipse.jetty.io.SelectorManager.doFinishConnect(SelectorManager.java:355)
	at shaded.v9_4.org.eclipse.jetty.io.ManagedSelector.processConnect(ManagedSelector.java:232)
	... 11 more

Have tried the code above twice, cluster times out after 2 hours and shuts down. 
However, ML Flow managed to log the previous runs, so we can continue 
from where the cluster shut down.

In [27]:
for depth in np.arange(9, 11):
  for learning_rate in [0.3, 0.5, 0.7, 0.9]:
    for feature_no in ['sqrt', None]:
      params_gb_w_q1_smote = {'loss': 'deviance',
                              'learning_rate': learning_rate,
                              'n_estimators': 1000,
                              'max_depth': depth,
                              'max_features': feature_no,
                              'random_state': 123
                             }
      log_gb('gb_w_q1_smote',
             params_gb_w_q1_smote,
             pd_train_w_q1_smote,
             pd_test_w_q1_4_predict,
             pd_test_w_q1_working)

precision: 0.1678832116788321
precision: 0.12408759124087591
precision: 0.08838383838383838
precision: 0.15328467153284672
precision: 0.1678832116788321
precision: 0.13157894736842105
precision: 0.09205020920502092
precision: 0.07692307692307693
precision: 0.1678832116788321
precision: 0.15328467153284672
precision: 0.1678832116788321
precision: 0.16058394160583941
precision: 0.0963855421686747
precision: 0.11678832116788321
precision: 0.077733860342556
precision: 0.16853932584269662

### No Resampling, w/o q1 data 

**RF**

* n_estimators - 1000
* max_depth - 1, 2, 3, 4, 5, 6, 7, 8, 9, 10
* max_features - 'sqrt', None
* random_state - 123

**GB**
* loss - 'deviance'
* learning_rate - 0.3, 0.5, 0.7, 0.9
* n_estimators - 1000
* max_depth - 1, 2, 3, 4, 5, 6, 7, 8, 9, 10
* max_features - 'sqrt', None
* random_state - 123

In [29]:
for depth in np.arange(1, 11):
  for feature_no in ['sqrt', None]:
    params_rf_wo_q1_wo_resamp = {'n_estimators': 1000,
                                 'max_depth': depth,
                                 'max_features': feature_no,
                                 'random_state': 123
                                }
    log_rf('rf_wo_q1_wo_resamp',
           params_rf_wo_q1_wo_resamp,
           pd_train_wo_q1_wo_resamp,
           pd_test_wo_q1_4_predict,
           pd_test_wo_q1_working)

precision: 0.1966019417475728
precision: 0.1882998171846435
precision: 0.10948905109489052
precision: 0.17518248175182483
precision: 0.11678832116788321
precision: 0.16058394160583941
precision: 0.12408759124087591
precision: 0.20437956204379562
precision: 0.15328467153284672
precision: 0.19708029197080293
precision: 0.1386861313868613
precision: 0.145985401459854
precision: 0.12408759124087591
precision: 0.12408759124087591
precision: 0.10948905109489052
precision: 0.12408759124087591
precision: 0.12408759124087591
precision: 0.12408759124087591
precision: 0.13138686131386862
precision: 0.13138686131386862

In [30]:
for depth in np.arange(1, 11):
  for learning_rate in [0.3, 0.5, 0.7, 0.9]:
    for feature_no in ['sqrt', None]:
      params_gb_wo_q1_wo_resamp = {'loss': 'deviance',
                                   'learning_rate': learning_rate,
                                   'n_estimators': 1000,
                                   'max_depth': depth,
                                   'max_features': feature_no,
                                   'random_state': 123
                                  }
      log_gb('gb_wo_q1_wo_resamp',
             params_gb_wo_q1_wo_resamp,
             pd_train_wo_q1_wo_resamp,
             pd_test_wo_q1_4_predict,
             pd_test_wo_q1_working)

precision: 0.10948905109489052
precision: 0.10948905109489052
precision: 0.145985401459854
precision: 0.13138686131386862
precision: 0.16058394160583941
precision: 0.145985401459854
precision: 0.15328467153284672
precision: 0.176759410801964
precision: 0.19708029197080293
precision: 0.10948905109489052
precision: 0.18248175182481752
precision: 0.2116788321167883
precision: 0.19708029197080293
precision: 0.18248175182481752
precision: 0.16058394160583941
precision: 0.17518248175182483
precision: 0.16058394160583941
precision: 0.12408759124087591
precision: 0.13138686131386862
precision: 0.15328467153284672
precision: 0.15328467153284672
precision: 0.15328467153284672
precision: 0.04660670482420278
precision: 0.10218978102189781
precision: 0.10948905109489052
precision: 0.12408759124087591
precision: 0.10218978102189781
precision: 0.16058394160583941
precision: 0.16058394160583941
precision: 0.10218978102189781
precision: 0.13274336283185842
precision: 0.15328467153284672
precision: 0.10218978102189781
precision: 0.10948905109489052
precision: 0.05154639175257732
precision: 0.17518248175182483
precision: 0.13138686131386862
precision: 0.1897810218978102
precision: 0.05570291777188329
precision: 0.12408759124087591
precision: 0.09141274238227147
precision: 0.10948905109489052
precision: 0.02021563342318059
precision: 0.06748466257668712
precision: 0.04176904176904177
precision: 0.072992700729927
precision: 0.13851992409867173
precision: 0.17518248175182483
precision: 0.13138686131386862
precision: 0.10948905109489052
precision: 0.07479674796747968
precision: 0.11678832116788321
precision: 0.10997067448680352
precision: 0.07194244604316546
precision: 0.03656597774244833
precision: 0.1678832116788321
precision: 0.10948905109489052
precision: 0.10948905109489052
precision: 0.050955414012738856
precision: 0.11678832116788321
precision: 0.09888888888888889
precision: 0.05517241379310345
precision: 0.07868383404864092
precision: 0.18435754189944134
precision: 0.11678832116788321
precision: 0.11678832116788321
precision: 0.11678832116788321
precision: 0.12408759124087591
precision: 0.05500705218617771
precision: 0.042105263157894736
precision: 0.07890743550834597
precision: 0.07536764705882353
precision: 0.11678832116788321
precision: 0.145985401459854
precision: 0.06563039723661486
precision: 0.1386861313868613
precision: 0.05970149253731343
precision: 0.11678832116788321
precision: 0.057947019867549666
precision: 0.08068459657701711

### Undersampling, w/o q1 data 

**RF**

* n_estimators - 1000
* max_depth - 1, 2, 3, 4, 5, 6, 7, 8, 9, 10
* max_features - 'sqrt', None
* random_state - 123

**GB**
* loss - 'deviance'
* learning_rate - 0.3, 0.5, 0.7, 0.9
* n_estimators - 1000
* max_depth - 1, 2, 3, 4, 5, 6, 7, 8, 9, 10
* max_features - 'sqrt', None
* random_state - 123

In [32]:
for depth in np.arange(1, 11):
  for feature_no in ['sqrt', None]:
    params_rf_wo_q1_undersamp = {'n_estimators': 1000,
                                 'max_depth': depth,
                                 'max_features': feature_no,
                                 'random_state': 123
                                }
    log_rf('rf_wo_q1_undersamp',
           params_rf_wo_q1_undersamp,
           pd_train_wo_q1_undersamp,
           pd_test_wo_q1_4_predict,
           pd_test_wo_q1_working)

precision: 0.1590909090909091
precision: 0.146163215590743
precision: 0.1678832116788321
precision: 0.20512820512820512
precision: 0.15328467153284672
precision: 0.1678832116788321
precision: 0.19708029197080293
precision: 0.1678832116788321
precision: 0.20437956204379562
precision: 0.22627737226277372
precision: 0.21897810218978103
precision: 0.18248175182481752
precision: 0.24817518248175183
precision: 0.2116788321167883
precision: 0.19708029197080293
precision: 0.18248175182481752
precision: 0.18248175182481752
precision: 0.1678832116788321
precision: 0.1897810218978102
precision: 0.16058394160583941

In [33]:
for depth in np.arange(1, 11):
  for learning_rate in [0.3, 0.5, 0.7, 0.9]:
    for feature_no in ['sqrt', None]:
      params_gb_wo_q1_undersamp = {'loss': 'deviance',
                                   'learning_rate': learning_rate,
                                   'n_estimators': 1000,
                                   'max_depth': depth,
                                   'max_features': feature_no,
                                   'random_state': 123
                                  }
      log_gb('gb_wo_q1_undersamp',
             params_gb_wo_q1_undersamp,
             pd_train_wo_q1_undersamp,
             pd_test_wo_q1_4_predict,
             pd_test_wo_q1_working)

precision: 0.15328467153284672
precision: 0.1386861313868613
precision: 0.1678832116788321
precision: 0.16058394160583941
precision: 0.18248175182481752
precision: 0.18248175182481752
precision: 0.1386861313868613
precision: 0.17518248175182483
precision: 0.1897810218978102
precision: 0.17518248175182483
precision: 0.16058394160583941
precision: 0.145985401459854
precision: 0.145985401459854
precision: 0.21897810218978103
precision: 0.12408759124087591
precision: 0.18248175182481752
precision: 0.1386861313868613
precision: 0.08759124087591241
precision: 0.10218978102189781
precision: 0.17518248175182483
precision: 0.12408759124087591
precision: 0.10948905109489052
precision: 0.15328467153284672
precision: 0.145985401459854
precision: 0.15328467153284672
precision: 0.12408759124087591
precision: 0.1678832116788321
precision: 0.11678832116788321
precision: 0.12408759124087591
precision: 0.1678832116788321
precision: 0.11678832116788321
precision: 0.13138686131386862
precision: 0.1897810218978102
precision: 0.18248175182481752
precision: 0.1678832116788321
precision: 0.1678832116788321
precision: 0.16058394160583941
precision: 0.16058394160583941
precision: 0.12408759124087591
precision: 0.15942028985507245
precision: 0.1897810218978102
precision: 0.145985401459854
precision: 0.18705035971223022
precision: 0.16546762589928057
precision: 0.14084507042253522
precision: 0.1736111111111111
precision: 0.14465408805031446
precision: 0.147239263803681
precision: 0.14492753623188406
precision: 0.16666666666666666
precision: 0.1360544217687075
precision: 0.16875
precision: 0.1368421052631579
precision: 0.1417624521072797
precision: 0.1440329218106996
precision: 0.1497175141242938
precision: 0.16312056737588654
precision: 0.16083916083916083
precision: 0.13705583756345177
precision: 0.167192429022082
precision: 0.12937062937062938
precision: 0.16363636363636364
precision: 0.12903225806451613
precision: 0.15384615384615385
precision: 0.1346153846153846
precision: 0.15816326530612246
precision: 0.15331010452961671
precision: 0.15057915057915058
precision: 0.13452914798206278
precision: 0.18154761904761904
precision: 0.1700404858299595
precision: 0.16774193548387098
precision: 0.16145833333333334
precision: 0.1449814126394052
precision: 0.1743119266055046
precision: 0.176056338028169
precision: 0.14678899082568808
precision: 0.17133956386292834
precision: 0.13744075829383887
precision: 0.13025210084033614

### Oversampling, w/o q1 data 

**RF**

* n_estimators - 1000
* max_depth - 1, 2, 3, 4, 5, 6, 7, 8, 9, 10
* max_features - 'sqrt', None
* random_state - 123

**GB**
* loss - 'deviance'
* learning_rate - 0.3, 0.5, 0.7, 0.9
* n_estimators - 1000
* max_depth - 1, 2, 3, 4, 5, 6, 7, 8, 9, 10
* max_features - 'sqrt', None
* random_state - 123

In [35]:
for depth in np.arange(1, 11):
  for feature_no in ['sqrt', None]:
    params_rf_wo_q1_oversamp = {'n_estimators': 1000,
                               'max_depth': depth,
                               'max_features': feature_no,
                               'random_state': 123
                              }
    log_rf('rf_wo_q1_oversamp',
           params_rf_wo_q1_oversamp,
           pd_train_wo_q1_oversamp,
           pd_test_wo_q1_4_predict,
           pd_test_wo_q1_working)

precision: 0.13154960981047936
precision: 0.10633116883116883
precision: 0.20625
precision: 0.1882998171846435
precision: 0.15328467153284672
precision: 0.19405940594059407
precision: 0.16058394160583941
precision: 0.22289156626506024
precision: 0.145985401459854
precision: 0.2088607594936709
precision: 0.17518248175182483
precision: 0.2116788321167883
precision: 0.16058394160583941
precision: 0.145985401459854
precision: 0.145985401459854
precision: 0.1678832116788321
precision: 0.145985401459854
precision: 0.16058394160583941
precision: 0.1386861313868613
precision: 0.15328467153284672

In [36]:
for depth in np.arange(1, 11):
  for learning_rate in [0.3, 0.5, 0.7, 0.9]:
    for feature_no in ['sqrt', None]:
      params_gb_wo_q1_oversamp = {'loss': 'deviance',
                                 'learning_rate': learning_rate,
                                 'n_estimators': 1000,
                                 'max_depth': depth,
                                 'max_features': feature_no,
                                 'random_state': 123
                                }
      log_gb('gb_wo_q1_oversamp',
             params_gb_wo_q1_oversamp,
             pd_train_wo_q1_oversamp,
             pd_test_wo_q1_4_predict,
             pd_test_wo_q1_working)

precision: 0.18248175182481752
precision: 0.18248175182481752
precision: 0.1897810218978102
precision: 0.17518248175182483
precision: 0.1897810218978102
precision: 0.1897810218978102
precision: 0.18248175182481752
precision: 0.18248175182481752
precision: 0.12408759124087591
precision: 0.13138686131386862
precision: 0.145985401459854
precision: 0.15328467153284672
precision: 0.15328467153284672
precision: 0.16058394160583941
precision: 0.16058394160583941
precision: 0.13138686131386862
precision: 0.16058394160583941
precision: 0.145985401459854
precision: 0.12408759124087591
precision: 0.10948905109489052
precision: 0.17518248175182483
precision: 0.1386861313868613
precision: 0.15328467153284672
precision: 0.17518248175182483
precision: 0.15328467153284672
precision: 0.12408759124087591
precision: 0.15328467153284672
precision: 0.13138686131386862
precision: 0.1897810218978102
precision: 0.16058394160583941
precision: 0.13138686131386862
precision: 0.08759124087591241
precision: 0.10218978102189781
precision: 0.10948905109489052
precision: 0.08759124087591241
precision: 0.072992700729927
precision: 0.08759124087591241
precision: 0.08029197080291971
precision: 0.08759124087591241
precision: 0.10948905109489052
precision: 0.12408759124087591
precision: 0.08759124087591241
precision: 0.0948905109489051
precision: 0.08029197080291971
precision: 0.1678832116788321
precision: 0.0948905109489051
precision: 0.10948905109489052
precision: 0.10218978102189781
precision: 0.0948905109489051
precision: 0.10218978102189781
precision: 0.10218978102189781
precision: 0.12408759124087591
precision: 0.10948905109489052
precision: 0.08759124087591241
precision: 0.08759124087591241
precision: 0.12408759124087591
precision: 0.11678832116788321
precision: 0.11678832116788321
precision: 0.10218978102189781
precision: 0.11678832116788321
precision: 0.12408759124087591
precision: 0.1386861313868613
precision: 0.12408759124087591
precision: 0.11678832116788321
precision: 0.13138686131386862
precision: 0.1386861313868613
precision: 0.12408759124087591
precision: 0.13138686131386862
precision: 0.12408759124087591
precision: 0.145985401459854
precision: 0.10218978102189781
precision: 0.1386861313868613
precision: 0.1386861313868613
precision: 0.1386861313868613
precision: 0.12408759124087591
precision: 0.1386861313868613
precision: 0.145985401459854
precision: 0.145985401459854
precision: 0.12408759124087591
precision: 0.11678832116788321

### SMOTE, w/o q1 data 

**RF**

* n_estimators - 1000
* max_depth - 1, 2, 3, 4, 5, 6, 7, 8, 9, 10
* max_features - 'sqrt', None
* random_state - 123

**GB**
* loss - 'deviance'
* learning_rate - 0.3, 0.5, 0.7, 0.9
* n_estimators - 1000
* max_depth - 1, 2, 3, 4, 5, 6, 7, 8, 9, 10
* max_features - 'sqrt', None
* random_state - 123

In [38]:
for depth in np.arange(1, 11):
  for feature_no in ['sqrt', None]:
    params_rf_wo_q1_smote = {'n_estimators': 1000,
                            'max_depth': depth,
                            'max_features': feature_no,
                            'random_state': 123
                           }
    log_rf('rf_wo_q1_smote',
           params_rf_wo_q1_smote,
           pd_train_wo_q1_smote,
           pd_test_wo_q1_4_predict,
           pd_test_wo_q1_working)

precision: 0.13836477987421383
precision: 0.11506849315068493
precision: 0.1794871794871795
precision: 0.1319052987598647
precision: 0.1878453038674033
precision: 0.1319052987598647
precision: 0.13138686131386862
precision: 0.1891891891891892
precision: 0.1386861313868613
precision: 0.1891891891891892
precision: 0.1386861313868613
precision: 0.2033195020746888
precision: 0.145985401459854
precision: 0.11678832116788321
precision: 0.13138686131386862
precision: 0.12408759124087591
precision: 0.1386861313868613
precision: 0.12408759124087591
precision: 0.12408759124087591
precision: 0.13138686131386862

In [39]:
for depth in np.arange(1, 11):
  for learning_rate in [0.3, 0.5, 0.7, 0.9]:
    for feature_no in ['sqrt', None]:
      params_gb_wo_q1_smote = {'loss': 'deviance',
                              'learning_rate': learning_rate,
                              'n_estimators': 1000,
                              'max_depth': depth,
                              'max_features': feature_no,
                              'random_state': 123
                             }
      log_gb('gb_wo_q1_smote',
             params_gb_wo_q1_smote,
             pd_train_wo_q1_smote,
             pd_test_wo_q1_4_predict,
             pd_test_wo_q1_working)

precision: 0.11678832116788321
precision: 0.12408759124087591
precision: 0.11678832116788321
precision: 0.10948905109489052
precision: 0.15328467153284672
precision: 0.13138686131386862
precision: 0.15328467153284672
precision: 0.17518248175182483
precision: 0.20437956204379562
precision: 0.17518248175182483
precision: 0.1897810218978102
precision: 0.12408759124087591
precision: 0.1897810218978102
precision: 0.12408759124087591
precision: 0.16058394160583941
precision: 0.13138686131386862
precision: 0.19708029197080293
precision: 0.12408759124087591
precision: 0.15328467153284672
precision: 0.13138686131386862
precision: 0.21897810218978103
precision: 0.1678832116788321
precision: 0.20437956204379562
precision: 0.01834862385321101
precision: 0.15328467153284672
precision: 0.12408759124087591
precision: 0.1678832116788321
precision: 0.13138686131386862
precision: 0.12408759124087591
precision: 0.18248175182481752
precision: 0.007670850767085077
precision: 0.145985401459854
precision: 0.1678832116788321
precision: 0.145985401459854
precision: 0.13138686131386862
precision: 0.1386861313868613
precision: 0.0841799709724238
precision: 0.11678832116788321
precision: 0.07805429864253394
precision: 0.1897810218978102
precision: 0.18248175182481752
precision: 0.10218978102189781
precision: 0.012706480304955527
precision: 0.11678832116788321
precision: 0.16058394160583941
precision: 0.03482587064676617
precision: 0.08922558922558922
precision: 0.08029197080291971
precision: 0.050970873786407765
precision: 0.10948905109489052
precision: 0.20437956204379562
precision: 0.08029197080291971
precision: 0.09324324324324325
precision: 0.06578947368421052
precision: 0.09467989179440937
precision: 0.10273972602739725
precision: 0.1386861313868613
precision: 0.16058394160583941
precision: 0.02929427430093209
precision: 0.10218978102189781
precision: 0.03793103448275862
precision: 0.0948905109489051
precision: 0.04074585635359116
precision: 0.003389830508474576
precision: 0.145985401459854
precision: 0.145985401459854
precision: 0.145985401459854
precision: 0.19708029197080293
precision: 0.05233111322549953
precision: 0.0
precision: 0.04515418502202643
precision: 0.03125
precision: 0.1678832116788321
precision: 0.145985401459854
precision: 0.12408759124087591
precision: 0.13138686131386862
precision: 0.10948905109489052
precision: 0.16058394160583941
precision: 0.09497206703910614
precision: 0.1386861313868613